# Data - Holdings

Analysis of the structure, availability and quality of holdings data from different ETF-providers.

## Outline

* Define and import a selection of ETFs holdings from different providers
* Analyze available columns
* Compare naming conventions of different data colunns over different providers

In [1]:
# General
import pandas as pd

# Hidden configurations
from mySecrets import config_file, AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_BUCKET

# Custom modules
import Tools

In [2]:
# Read config table for (ISIN / Provider) mapping
config = pd.read_excel(config_file, sheet_name="ETF_Overview", header=1)
config = config[['Security_ISIN', 'Fund_Company', 'Type']]
config.head()

,Security_ISIN,Fund_Company,Type
0,LU0836512706,iShares,Aktien
1,LU0836513852,iShares,Anleihen
2,LU0836512615,iShares,Aktien
3,LU0836513266,iShares,Anleihen
4,LU0836512888,iShares,Aktien


In [1]:
# Connect to AWS S3 storage
s3 = Tools.S3()
s3.connect(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)

In [5]:
# Download all position files
file_list = s3.listFiles(AWS_BUCKET, "data/position_trans/2022_03_04")

# Filter to files ending in .tsv
file_list = [i for i in file_list if '.tsv' in i]

positions = pd.DataFrame(columns = ['ISIN_FUND', 'ISIN', 'NAME', 'WEIGHT', 'TICKER', 'SECTOR', 'EXCHANGE', 'COUNTRY'])

for key in file_list:

    data = s3.readFile(AWS_BUCKET, key, sep = "\t")  

    # Subset columns
    cols = ['ISIN_FUND', 'ISIN', 'NAME', 'WEIGHT', 'TICKER', 'SECTOR', 'EXCHANGE', 'COUNTRY']
    data = data[[i for i in cols if i in data.columns]]

    # Append 
    positions = positions.append(data)

# Store in data format readable by both Python and R
positions = positions.reset_index().drop(['index'], axis = 1)

positions.head()

,ISIN_FUND,ISIN,NAME,WEIGHT,TICKER,SECTOR,EXCHANGE,COUNTRY
0,DE0002635265,DE000HV2AS10,UNICREDIT BANK AG RegS,1.01,HVB,Gedeckt,-,Deutschland
1,DE0002635265,DE000A1KRJV6,ING DIBA AG RegS,0.97,INGDIB,Gedeckt,-,Deutschland
2,DE0002635265,DE000CZ45VS1,COMMERZBANK AG MTN RegS,0.88,CMZB,Gedeckt,-,Deutschland
3,DE0002635265,XS2106576494,LANDESBANK HESSEN THUERINGEN GIROZ MTN RegS,0.84,HESLAN,Gedeckt,-,Deutschland
4,DE0002635265,XS1767931477,LANDESBANK HESSEN THUERINGEN GIROZ RegS,0.83,HESLAN,Gedeckt,-,Deutschland


In [15]:
# Analyze naming conventions
positions[positions.NAME.str.contains('Microsoft', na = False, case = False)].NAME.unique()

array(['MICROSOFT CORP', 'MICROSOFT CORPORATION', 'MICROSOFT-T ORD',
       'MICROSOFT CORP 08/26 594918BR', 'MICROSOFT CORP 11/25 594918BJ',
       'MICROSOFT CORP 06/27 594918BY', 'MICROSOFT CORP 02/25 594918BB',
       'MICROSOFT CORP 06/24 594918BX', 'MICROSOFT CORP 12/23',
       'MICROSOFT CORP 08/23 594918BQ', 'MICROSOFT CORP 05/23',
       'MICROSOFT CORP 11/22 594918BH', 'MICROSOFT CORP 11/22',
       'BBG00ZK0QMV8 MICROSOFT CORPORATION 3/52',
       'MICROSOFT COR 2.525% JUN50 6/50',
       'MICROSOFT COR 2.675% JUN60 6/60', 'MICROSOFT CORP 08/46 594918BT',
       'MICROSOFT CORP 08/36 594918BS',
       'BBG00ZK0RSR9 MICROSOFT CORP 00000 62 3/62',
       'MICROSOFT CORP 02/35 594918BC', 'MICROSOFT CORP 06/47 594918CA',
       'MICROSOFT CORP 11/45 594918BL', 'MICROSOFT CORP 06/37 594918BZ',
       'MICROSOFT CORP 06/57 594918CB'], dtype=object)

In [21]:
# Names of countries
positions[positions.NAME.str.contains('Microsoft', na = False, case = False)].COUNTRY.unique()

array(['Vereinigte Staaten', 'USA', 'Vereinigte Staaten von Amerika'],
      dtype=object)

In [22]:
# Names of sectors / industries
positions[positions.NAME.str.contains('Microsoft', na = False, case = False)].SECTOR.unique()

array(['IT', 'Information Technology', 'Technologie', 'Unternehmen',
       'Industrie', 'Informationstechnologie', 'unknown', 'Technology',
       'System-Software'], dtype=object)